<a href="https://colab.research.google.com/github/omarhashem80/used-cars-price/blob/main/used_cars_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#! pip install category_encoders

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Lasso, Ridge ,ElasticNet
from sklearn.tree import  DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.pipeline import Pipeline
from category_encoders import BinaryEncoder
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.compose import ColumnTransformer
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('./train-data.csv')
df

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6014,6014,Maruti Swift VDI,Delhi,2014,27365,Diesel,Manual,First,28.4 kmpl,1248 CC,74 bhp,5.0,7.88 Lakh,4.75
6015,6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,Diesel,Manual,First,24.4 kmpl,1120 CC,71 bhp,5.0,NaN,4.00
6016,6016,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Diesel,Manual,Second,14.0 kmpl,2498 CC,112 bhp,8.0,NaN,2.90
6017,6017,Maruti Wagon R VXI,Kolkata,2013,46000,Petrol,Manual,First,18.9 kmpl,998 CC,67.1 bhp,5.0,NaN,2.65


In [5]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               6019 non-null   object 
 1   Location           6019 non-null   object 
 2   Year               6019 non-null   int64  
 3   Kilometers_Driven  6019 non-null   int64  
 4   Fuel_Type          6019 non-null   object 
 5   Transmission       6019 non-null   object 
 6   Owner_Type         6019 non-null   object 
 7   Mileage            6017 non-null   object 
 8   Engine             5983 non-null   object 
 9   Power              5983 non-null   object 
 10  Seats              5977 non-null   float64
 11  New_Price          824 non-null    object 
 12  Price              6019 non-null   float64
dtypes: float64(2), int64(2), object(9)
memory usage: 611.4+ KB


In [7]:
df.duplicated().sum()

0

In [8]:
df.isna().mean()*100

Name                  0.000000
Location              0.000000
Year                  0.000000
Kilometers_Driven     0.000000
Fuel_Type             0.000000
Transmission          0.000000
Owner_Type            0.000000
Mileage               0.033228
Engine                0.598106
Power                 0.598106
Seats                 0.697790
New_Price            86.310018
Price                 0.000000
dtype: float64

In [9]:
df.drop('New_Price',axis=1,inplace=True)

In [10]:
df.dropna(inplace=True)

In [11]:
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74


In [12]:
df.Name[:20]

0                               Maruti Wagon R LXI CNG
1                     Hyundai Creta 1.6 CRDi SX Option
2                                         Honda Jazz V
3                                    Maruti Ertiga VDI
4                      Audi A4 New 2.0 TDI Multitronic
5                      Hyundai EON LPG Era Plus Option
6                               Nissan Micra Diesel XV
7                    Toyota Innova Crysta 2.8 GX AT 8S
8                  Volkswagen Vento Diesel Comfortline
9                       Tata Indica Vista Quadrajet LS
10                                    Maruti Ciaz Zeta
11                         Honda City 1.5 V AT Sunroof
12                               Maruti Swift VDI BSIV
13                    Land Rover Range Rover 2.2L Pure
14                      Land Rover Freelander 2 TD4 SE
15                         Mitsubishi Pajero Sport 4X4
16                               Honda Amaze S i-Dtech
17                               Maruti Swift DDiS VDI
18        

In [13]:
df.Name.nunique()

1855

In [14]:
df['Brand'] = df.Name.str.split(' ').str[0]
df['Model'] = df.Name.str.split(' ').str[1]

df.drop('Name', axis=1, inplace=True)

In [15]:
cats = df.select_dtypes(include='O').columns
cats

Index(['Location', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Mileage',
       'Engine', 'Power', 'Brand', 'Model'],
      dtype='object')

In [16]:
for cat in cats:
    print(cat,df[cat].value_counts(),'*'*50,sep='\n')

Location
Mumbai        784
Hyderabad     741
Kochi         648
Coimbatore    634
Pune          613
Delhi         549
Kolkata       530
Chennai       490
Jaipur        410
Bangalore     353
Ahmedabad     223
Name: Location, dtype: int64
**************************************************
Fuel_Type
Diesel    3195
Petrol    2714
CNG         56
LPG         10
Name: Fuel_Type, dtype: int64
**************************************************
Transmission
Manual       4266
Automatic    1709
Name: Transmission, dtype: int64
**************************************************
Owner_Type
First             4903
Second             953
Third              111
Fourth & Above       8
Name: Owner_Type, dtype: int64
**************************************************
Mileage
18.9 kmpl     172
17.0 kmpl     170
18.6 kmpl     119
20.36 kmpl     88
21.1 kmpl      86
             ... 
14.57 kmpl      1
14.33 kmpl      1
22.8 km/kg      1
8.0 kmpl        1
17.24 kmpl      1
Name: Mileage, Length: 442, dtype: int

In [17]:
df.Owner_Type = df.Owner_Type.map({'First':1,'Second':2,'Third':3,'Fourth & Above':4})
df.Engine = df.Engine.str.split(' ').str[0]
df.Power = df.Power.str.split(' ').str[0]

In [18]:
df

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Brand,Model
0,Mumbai,2010,72000,CNG,Manual,1,26.6 km/kg,998,58.16,5.0,1.75,Maruti,Wagon
1,Pune,2015,41000,Diesel,Manual,1,19.67 kmpl,1582,126.2,5.0,12.50,Hyundai,Creta
2,Chennai,2011,46000,Petrol,Manual,1,18.2 kmpl,1199,88.7,5.0,4.50,Honda,Jazz
3,Chennai,2012,87000,Diesel,Manual,1,20.77 kmpl,1248,88.76,7.0,6.00,Maruti,Ertiga
4,Coimbatore,2013,40670,Diesel,Automatic,2,15.2 kmpl,1968,140.8,5.0,17.74,Audi,A4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6014,Delhi,2014,27365,Diesel,Manual,1,28.4 kmpl,1248,74,5.0,4.75,Maruti,Swift
6015,Jaipur,2015,100000,Diesel,Manual,1,24.4 kmpl,1120,71,5.0,4.00,Hyundai,Xcent
6016,Jaipur,2012,55000,Diesel,Manual,2,14.0 kmpl,2498,112,8.0,2.90,Mahindra,Xylo
6017,Kolkata,2013,46000,Petrol,Manual,1,18.9 kmpl,998,67.1,5.0,2.65,Maruti,Wagon


In [19]:
def get_mileage(x):
    num,unit = x.split(' ')
    num = float(num)
    if unit == 'km/kg':
        return num
    elif unit == 'kmpl':
        return num*1.61
    return np.nan

In [20]:
df.Mileage = df.Mileage.apply(get_mileage)

In [21]:
df

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Brand,Model
0,Mumbai,2010,72000,CNG,Manual,1,26.6000,998,58.16,5.0,1.75,Maruti,Wagon
1,Pune,2015,41000,Diesel,Manual,1,31.6687,1582,126.2,5.0,12.50,Hyundai,Creta
2,Chennai,2011,46000,Petrol,Manual,1,29.3020,1199,88.7,5.0,4.50,Honda,Jazz
3,Chennai,2012,87000,Diesel,Manual,1,33.4397,1248,88.76,7.0,6.00,Maruti,Ertiga
4,Coimbatore,2013,40670,Diesel,Automatic,2,24.4720,1968,140.8,5.0,17.74,Audi,A4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6014,Delhi,2014,27365,Diesel,Manual,1,45.7240,1248,74,5.0,4.75,Maruti,Swift
6015,Jaipur,2015,100000,Diesel,Manual,1,39.2840,1120,71,5.0,4.00,Hyundai,Xcent
6016,Jaipur,2012,55000,Diesel,Manual,2,22.5400,2498,112,8.0,2.90,Mahindra,Xylo
6017,Kolkata,2013,46000,Petrol,Manual,1,30.4290,998,67.1,5.0,2.65,Maruti,Wagon


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5975 entries, 0 to 6018
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Location           5975 non-null   object 
 1   Year               5975 non-null   int64  
 2   Kilometers_Driven  5975 non-null   int64  
 3   Fuel_Type          5975 non-null   object 
 4   Transmission       5975 non-null   object 
 5   Owner_Type         5975 non-null   int64  
 6   Mileage            5975 non-null   float64
 7   Engine             5975 non-null   object 
 8   Power              5975 non-null   object 
 9   Seats              5975 non-null   float64
 10  Price              5975 non-null   float64
 11  Brand              5975 non-null   object 
 12  Model              5975 non-null   object 
dtypes: float64(3), int64(3), object(7)
memory usage: 653.5+ KB


In [23]:
df.Power = pd.to_numeric(df.Power, errors='coerce')
df.Engine = pd.to_numeric(df.Engine, errors='coerce')

In [24]:
df.isna().mean()*100

Location             0.000000
Year                 0.000000
Kilometers_Driven    0.000000
Fuel_Type            0.000000
Transmission         0.000000
Owner_Type           0.000000
Mileage              0.000000
Engine               0.000000
Power                1.723849
Seats                0.000000
Price                0.000000
Brand                0.000000
Model                0.000000
dtype: float64

In [25]:
nums = df.describe().columns
df.describe()

,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price
count,5975.000000,5.975000e+03,5975.000000,5975.000000,5975.000000,5872.000000,5975.000000,5975.000000
mean,2013.386778,5.867431e+04,1.200669,29.103737,1621.606695,113.276894,5.278828,9.501647
std,3.247238,9.155851e+04,0.453458,7.169916,601.036987,53.881892,0.808959,11.205736
min,1998.000000,1.710000e+02,1.000000,0.000000,624.000000,34.200000,0.000000,0.440000
25%,2012.000000,3.390800e+04,1.000000,24.311000,1198.000000,75.000000,5.000000,3.500000
50%,2014.000000,5.300000e+04,1.000000,29.141000,1493.000000,97.700000,5.000000,5.650000
75%,2016.000000,7.300000e+04,1.000000,33.681200,1984.000000,138.100000,5.000000,9.950000
max,2019.000000,6.500000e+06,4.000000,45.724000,5998.000000,560.000000,10.000000,160.000000


In [26]:
px.box(df,nums[0])

In [27]:
px.box(df,nums[1])

In [28]:
px.box(df,nums[2])

In [29]:
px.box(df,nums[3])

In [30]:
px.box(df,nums[4])

In [31]:
px.box(df,nums[5])

In [32]:
px.box(df,nums[6])

In [33]:
px.box(df,nums[7])

In [34]:
df.describe(include='object')

,Location,Fuel_Type,Transmission,Brand,Model
count,5975,5975,5975,5975,5975
unique,11,4,2,31,209
top,Mumbai,Diesel,Manual,Maruti,Swift
freq,784,3195,4266,1197,343


In [35]:
px.imshow(df.corr(),text_auto='.2f')

In [36]:
df.quantile([.7,.75,.8,.85,.9,.95,.99])

,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price
0.70,2015.0,68344.2,1.0,32.7796,1799.00,126.20,5.0,8.4500
0.75,2016.0,73000.0,1.0,33.6812,1984.00,138.10,5.0,9.9500
0.80,2016.0,79000.0,1.0,35.4200,2143.00,153.86,5.0,12.5000
0.85,2017.0,85286.4,2.0,36.6114,2179.00,171.00,5.0,16.5000
0.90,2017.0,96719.0,2.0,38.3824,2494.00,184.00,7.0,22.5000
0.95,2018.0,120000.0,2.0,40.8940,2982.00,204.00,7.0,32.5000
0.99,2019.0,172297.0,3.0,45.2249,3197.26,287.22,8.0,56.0364


In [37]:
df.drop(df[df['Kilometers_Driven'] > 100000].index, axis = 0, inplace = True)
df.drop(df[df['Power'] > 480].index, axis = 0, inplace = True)

In [38]:
X = df.drop('Price',axis=1)
y = df['Price'].values

In [39]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [40]:
num_transformer = Pipeline(steps = [
         ('impute',SimpleImputer(strategy='median')),
         ('scale',StandardScaler()),
      ])
cat_transformer = Pipeline(steps = [('encode',BinaryEncoder())])

preprocessor= ColumnTransformer(transformers=[
    ('num',num_transformer,['Year', 'Kilometers_Driven', 'Mileage', 'Engine', 'Power', 'Seats']),
    ('cat',cat_transformer,['Location', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Model']),
])
X_trainp = preprocessor.fit_transform(X_train)
X_testp = preprocessor.transform(X_test)

In [41]:
X_trainp

array([[ 1.08136343, -1.44039125,  1.29484605, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.12070489, -0.57582942, -0.46018416, ...,  0.        ,
         1.        ,  0.        ],
       [-2.12083172,  1.0687553 , -0.40045801, ...,  0.        ,
         1.        ,  1.        ],
       ...,
       [-1.80061221,  0.03936184, -0.27641137, ...,  0.        ,
         1.        ,  1.        ],
       [ 0.12070489,  0.2967102 ,  1.03296981, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.12070489,  1.96947457,  1.19377101, ...,  0.        ,
         1.        ,  1.        ]])

In [42]:
# Define models to evaluate
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'Support Vector Machine': SVR(),
    'Decision Tree': DecisionTreeRegressor(),
    'K-Nearest Neighbors': KNeighborsRegressor(),
}

# Define hyperparameters for each model
params = {
    'Linear Regression': {
    },
    'Random Forest': {
        'regressor__n_estimators': [100, 200, 300],
        'regressor__max_depth': [None, 5, 10]
    },
    'Support Vector Machine': {'regressor__kernel': ['linear', 'rbf'], 'regressor__C': [0.1, 1, 10]},
    'Decision Tree': {'regressor__max_depth': [None, 5, 10]},
    'K-Nearest Neighbors': {'regressor__n_neighbors': [3, 5, 7]},
}

# Perform model selection and evaluation
results = {}
for model_name, model in models.items():
    # Construct a pipeline with scaler and regressor
    pipeline = Pipeline([
        ('regressor', model)
    ])
    print(model_name)
    # Perform hyperparameter tuning with cross-validation
    grid_search = GridSearchCV(pipeline, params[model_name], cv=5, scoring='neg_mean_squared_error', verbose=1)
    grid_search.fit(X_trainp, y_train)

    # Store results
    results[model_name] = {
        'Best Parameters': grid_search.best_params_,
        'Best Score': -grid_search.best_score_
    }

    # Perform cross-validation to estimate performance
    cv_scores = cross_val_score(pipeline, X_trainp, y_train, cv=5, scoring='neg_mean_squared_error')
    results[model_name]['CV Scores'] = -cv_scores  # Convert back to positive MSE

# Print results
for model_name, result in results.items():
    print(f"Model: {model_name}")
    print(f"Best Parameters: {result['Best Parameters']}")
    print(f"Best Score (MSE): {result['Best Score']}")
    print(f"Cross-Validation Scores (MSE): {result['CV Scores']}")
    print(f"Average Cross-Validation Score (MSE): {np.mean(result['CV Scores'])}\n")

# Select the best model based on average cross-validation score
best_model_name = min(results, key=lambda x: np.mean(results[x]['CV Scores']))
best_model = models[best_model_name]

# Train the best model on the full training data
best_model.fit(X_trainp, y_train)

# Evaluate the best model on the testing data
y_pred = best_model.predict(X_testp)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Testing Data: {mse}")
print(r2_score(y_test, y_pred))

Linear Regression
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Random Forest
Fitting 5 folds for each of 9 candidates, totalling 45 fits
Support Vector Machine
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Decision Tree
Fitting 5 folds for each of 3 candidates, totalling 15 fits
K-Nearest Neighbors
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Model: Linear Regression
Best Parameters: {}
Best Score (MSE): 36.87595228286534
Cross-Validation Scores (MSE): [30.23233223 35.77465113 35.27837392 30.10125055 52.99315358]
Average Cross-Validation Score (MSE): 36.87595228286534

Model: Random Forest
Best Parameters: {'regressor__max_depth': None, 'regressor__n_estimators': 300}
Best Score (MSE): 16.45318440398096
Cross-Validation Scores (MSE): [13.03251271 21.73021117 15.57646587 10.67209216 22.32550268]
Average Cross-Validation Score (MSE): 16.667356918330597

Model: Support Vector Machine
Best Parameters: {'regressor__C': 10, 'regressor__kernel': 'rbf

In [43]:
best_model

RandomForestRegressor()


In [44]:
from sklearn.preprocessing import PolynomialFeatures

# Define models to evaluate
models = {
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet Regression': ElasticNet()
}

# Define hyperparameters for each model
params = {
    'Ridge Regression': {'regressor__alpha': [0.1, 1, 10]},
    'Lasso Regression': {'regressor__alpha': [0.1, 1, 10]},
    'ElasticNet Regression': {'regressor__alpha': [0.1, 1, 10], 'regressor__l1_ratio': [0.1, 0.5, 0.9]}
}

# Perform model selection and evaluation
results = {}
for model_name, model in models.items():
    param_grid = {
        'poly__degree': [None, 2],  # Polynomial degrees to try
        **params[model_name]  # Hyperparameters for the model
    }

    # Construct a pipeline with polynomial features and regressor
    pipeline = Pipeline([
        ('log',np.log)
        ('poly', PolynomialFeatures()),
        ('regressor', model)
    ])
    print(model_name)
    # Perform hyperparameter tuning with cross-validation
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)
    grid_search.fit(X_trainp, y_train)

    # Store results
    results[model_name] = {
        'Best Parameters': grid_search.best_params_,
        'Best Score': -grid_search.best_score_
    }

    # Perform cross-validation to estimate performance
    cv_scores = cross_val_score(grid_search.best_estimator_, X_trainp, y_train, cv=5, scoring='neg_mean_squared_error')
    results[model_name]['CV Scores'] = -cv_scores  # Convert back to positive MSE

# Print results
for model_name, result in results.items():
    print(f"Model: {model_name}")
    print(f"Best Parameters: {result['Best Parameters']}")
    print(f"Best Score (MSE): {result['Best Score']}")
    print(f"Cross-Validation Scores (MSE): {result['CV Scores']}")
    print(f"Average Cross-Validation Score (MSE): {np.mean(result['CV Scores'])}\n")

# Select the best model based on average cross-validation score
best_model_name = min(results, key=lambda x: np.mean(results[x]['CV Scores']))
best_model = models[best_model_name]

# Train the best model on the full training data
best_model.fit(X_trainp, y_train)

# Evaluate the best model on the testing data
y_pred = best_model.predict(X_testp)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Testing Data: {mse}")
print(f"R-squared Score on Testing Data: {r2_score(y_test, y_pred)}")

Ridge Regression
Fitting 5 folds for each of 6 candidates, totalling 30 fits
Lasso Regression
Fitting 5 folds for each of 6 candidates, totalling 30 fits
ElasticNet Regression
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Model: Ridge Regression
Best Parameters: {'poly__degree': 2, 'regressor__alpha': 10}
Best Score (MSE): 16.75546488874888
Cross-Validation Scores (MSE): [12.66851946 21.62893162 12.55884345 12.33271222 24.58831769]
Average Cross-Validation Score (MSE): 16.75546488874888

Model: Lasso Regression
Best Parameters: {'poly__degree': 2, 'regressor__alpha': 0.1}
Best Score (MSE): 19.94450806363769
Cross-Validation Scores (MSE): [14.18989557 23.88907447 17.08496113 14.64878931 29.90981984]
Average Cross-Validation Score (MSE): 19.94450806363769

Model: ElasticNet Regression
Best Parameters: {'poly__degree': 2, 'regressor__alpha': 0.1, 'regressor__l1_ratio': 0.1}
Best Score (MSE): 18.586563800102397
Cross-Validation Scores (MSE): [13.49021504 22.12463691 15.45542